In [6]:
from google.colab import drive
drive.mount('/content/drive')
# Imports
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from numba import cuda

Mounted at /content/drive


In [2]:
def save_model(epoch, model, optimizer, loss, name):
  torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
  }, name+'.pth')

def load_model(model, optimizer, epoch, loss, name):
  checkpoint = torch.load(name+'.pth')
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  loss = checkpoint['loss']
  return model, optimizer, epoch, loss

def loadData(trainPath, testPath, batch_size):

  transformer = transforms.Compose([
    transforms.Resize((256,256)),  #
    #transforms.CenterCrop(224),  #
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
  ])

  train_loader = DataLoader(
      torchvision.datasets.ImageFolder(trainPath, transform = transformer),
      batch_size=batch_size,
      shuffle=True)

  test_loader = DataLoader(
      torchvision.datasets.ImageFolder(testPath, transform = transformer),
      batch_size=batch_size,
      shuffle=True)

  return train_loader, test_loader

In [10]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    #Canales de entrada, Canales de salida, Tamaño de Kernel, Stride y Padding
    self.conv1 = nn.Conv2d(3, 16, 3, 1, 0)
    self.conv2 = nn.Conv2d(16, 64, 3, 1, 0)
    self.conv3 = nn.Conv2d(64, 128, 3, 1, 0)
    self.dropout = nn.Dropout(0.2)
    self.fc1 = nn.Linear(128*30*30, 512)
    self.fc2 = nn.Linear(512, 256)

  def forward(self, x):
    #Entrada (batch, channels, ancho, largo)
    # (64, 3, 256, 256)
    x = torch.relu(self.conv1(x))
    x = torch.max_pool2d(x, 2, 2) #input, kernel_size, stride
    # (64, 16, 127, 127)
    x = torch.relu(self.conv2(x))
    x = torch.max_pool2d(x, 2, 2)
    # (64, 64, 62, 62)
    x = torch.relu(self.conv3(x))
    x = self.dropout(x)
    x = torch.max_pool2d(x, 2, 2)
    # (64, 128, 30, 30)
    x = x.view(-1, 128*30*30)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.softmax(x, dim=1)

    return x

In [4]:
#device = cuda.get_current_device()
#device.reset()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Is Torch Cuda Available? " + str(torch.cuda.is_available()))
print("Device: " + str(device))

Is Torch Cuda Available? True
Device: cuda


In [12]:
#trainPath = "Covid19-dataset/train"
#testPath = "Covid19-dataset/test"
trainPath = "/content/drive/My Drive/Colab Notebooks/Images/Covid19-datasetAumentado/train"
testPath = "/content/drive/My Drive/Colab Notebooks/Images/Covid19-datasetAumentado/test"
batch_size = 64
trainloader, testloader = loadData(trainPath, testPath, batch_size)

In [18]:
# Inicializar el modelo, la función de pérdida y el optimizador
model = CNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Entrenamiento del modelo
num_epochs = 25
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 5 == 4:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0

print('Finished Training')

[1,     5] loss: 5.329
[2,     5] loss: 5.277
[3,     5] loss: 5.277
[4,     5] loss: 5.255
[5,     5] loss: 5.258
[6,     5] loss: 5.252
[7,     5] loss: 5.264
[8,     5] loss: 5.271
[9,     5] loss: 5.280
[10,     5] loss: 5.277
[11,     5] loss: 5.271
[12,     5] loss: 5.264
[13,     5] loss: 5.296
[14,     5] loss: 5.252
[15,     5] loss: 5.302
[16,     5] loss: 5.280
[17,     5] loss: 5.274
[18,     5] loss: 5.277
[19,     5] loss: 5.249
[20,     5] loss: 5.267
[21,     5] loss: 5.280
[22,     5] loss: 5.274
[23,     5] loss: 5.283
[24,     5] loss: 5.264
[25,     5] loss: 5.249
Finished Training


In [14]:
# Evaluación del modelo
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test images: %d %%' % (100 * correct / total))

Accuracy on the test images: 30 %
